`Without RAG`

In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

query = "What did Keyukemi Ubi say about the year 2024?"

messages = [
    ("system", "You are a helpful assistant with access to relevant documents."),
    ("human", f"{query}"),
]

response = llm.invoke(messages)
print(response.content)

I apologize, but I couldn't find any information on a person named Keyukemi Ubi making a statement about the year 2024. It's possible that the person is not a public figure or didn't make a notable statement about the year 2024. Can you provide more context or clarify who Keyukemi Ubi is?


`With RAG`

In [2]:
from langchain_groq import ChatGroq
from langchain.vectorstores import Pinecone as pc
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

load_dotenv()

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

c:\Users\okwy_\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()

pinecone = Pinecone()

index_name = "default-index"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, 
        dimension=1536,
        spec=ServerlessSpec(
            cloud='aws', region='us-east-1'))

index = pinecone.Index(index_name)

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

documents = [
    (r"2024 A Diary Entry (That Probably Should Have Stayed in My Head) - VITU na VITU.txt", open(r"2024 A Diary Entry (That Probably Should Have Stayed in My Head) - VITU na VITU.txt", "r", encoding="utf-8").read(), {"author": "VITU na VITU"}),
    (r"Diary entries from Anhedonia - Keyukemi Ubi.txt", open(r"Diary entries from Anhedonia - Keyukemi Ubi.txt", "r", encoding="utf-8").read(), {"author": "Keyukemi Ubi"}),
]

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)

for doc_id, content, metadata in documents:
    chunks = text_splitter.split_text(content) 
    for i, chunk in enumerate(chunks):
        embedding = embeddings.embed_query(chunk)
        chunk_metadata = metadata.copy()
        chunk_metadata["chunk_index"] = i
        index.upsert([(f"{doc_id}_{i}", embedding, {"content": chunk, **chunk_metadata})])

In [8]:
query = "What did Vitu na Vitu say about the year 2024?"
query_embedding = embeddings.embed_query(query)
results = index.query(
    vector=query_embedding,
    top_k=6,
    include_metadata=True
)

retrieved_contents = [(result["metadata"]["author"],result["metadata"]["content"]) for result in results["matches"]]

messages = [
    ("system", "You are a helpful assistant with access to relevant documents."),
    ("human", f"The following documents may help:\n\n {retrieved_contents}.\n\n Now, {query}"),
]

response = llm.invoke(messages)
print(response.content)

According to the documents, Vitu na Vitu said:

"I can’t even begin to tell you how I feel about 2024. Not because it’s hard to articulate, but because part of my brain has chosen vibes over storage. Some memories are fried and deleted — like a phone app clearing space because too much trauma detected. It’s my brain’s way of saying, “That’s above my pay grade.”"

It seems that Vitu na Vitu is expressing a sense of overwhelm and trauma related to the year 2024, and is struggling to process their emotions about it.
